### Restart and Run All Cells

### Input files: Price-Today, orders-log
### Output files: Price-for-Order, orders-log, monthly-sales

In [3]:
import glob
import os
import pandas as pd
from datetime import date, timedelta, datetime
from pandas.tseries.offsets import *
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()
engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()
engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

pd.options.display.max_rows = 50
#pd.options.display.float_format = '{:.2f}'.format

today = date.today()
yesterday = today - timedelta(days=1)
print(today, yesterday)

2025-08-31 2025-08-30


In [4]:
# Get the current time
current_time = datetime.now()
# Format the time to remove milliseconds
fmt_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(fmt_time)

2025-08-31 17:01:35


In [5]:
current_path = os.getcwd()
base_path = os.path.dirname(current_path)
dat_path = os.path.join(base_path, "Data")
print(f"Data path (dat_path): {dat_path}") 
file_name = "Price-Today.csv"
input_file = os.path.join(dat_path, file_name)
print(f"Input file: {input_file}")

Data path (dat_path): C:\Users\PC1\OneDrive\A5\Data
Input file: C:\Users\PC1\OneDrive\A5\Data\Price-Today.csv


In [6]:
# Define a formatting function that converts a float to a string with two decimal places
def to_currency(x):
    return '฿{:,.2f}'.format(x)    

# Define a formatting function that converts an integer to a string with comma and no decimal places
def to_integer(x):
    return '{:,.0f}'.format(x)

# Define a formatting function that converts an integer to a string with comma and no decimal places
def to_percent(x):
    return '{:,.2f}%'.format(x)

# Define a formatting function that converts an integer to a string with comma and no decimal places
def to_date(x):
    return '{:%Y-%m-%d}'.format(x)

In [7]:
df_price_tdy = pd.read_csv(input_file)
df_price_tdy.dtypes
#df_price_tdy.columns.tolist()

name         object
date         object
price       float64
change      float64
%change     float64
open        float64
high        float64
low         float64
volume        int64
value(m)    float64
dtype: object

### Purchase recently sold stocks if price lower than last sold

In [9]:
#Select name from portfolio
sql = """SELECT name FROM buy"""
df_buy = pd.read_sql(sql, const)
names = df_buy.name
portfolio = ", ".join(map(lambda name: "'%s'" % name, names))
portfolio 

"'3BBIF', 'AH', 'AIMIRT', 'AWC', 'BCH', 'CPF', 'CPNREIT', 'DIF', 'GVREIT', 'IVL', 'JMART', 'JMT', 'MCS', 'NER', 'ORI', 'PTG', 'PTT', 'PTTGC', 'RCL', 'SCC', 'SENA', 'SINGER', 'STA', 'SYNEX', 'TFFIF', 'TOA', 'TVO', 'WHAIR', 'WHART'"

In [10]:
#
sql = """
    SELECT T.name, S.date AS sell_date, S.price AS sell_price, B.price AS buy_price, B.date AS buy_date, B.qty
    FROM sells S
    JOIN buys B
    ON S.buy_id = B.id 
    JOIN stocks T
    ON T.id = B.stock_id 
    WHERE T.name IN ({})
    ORDER BY T.name, S.date DESC
""".format(portfolio)
df_sells = pd.read_sql(sql, conpf)
df_sells.shape

(247, 6)

In [11]:
df_sells.dtypes
#df_sells.columns.tolist()

name           object
sell_date      object
sell_price    float64
buy_price     float64
buy_date       object
qty             int64
dtype: object

In [12]:
# Convert dates to datetime if they're not already
df_sells['sell_date'] = pd.to_datetime(df_sells['sell_date'])

# Get the latest record for each name
latest_records = df_sells.sort_values('sell_date', ascending=False).drop_duplicates('name')
latest_records.shape

(27, 6)

In [13]:
colu = 'name sell_date qty sell_price price low target %decline'.split()
sell_data   = pd.merge(latest_records, df_price_tdy, on='name', how='inner')
sell_data['diff'] = sell_data['price'] - sell_data['sell_price']
sell_data['%decline'] = round(sell_data['diff'] / (sell_data['sell_price']) * 100,2)
#sell_data['sell_date'] = pd.to_datetime(sell_data['sell_date'])
sell_data['target'] = round(sell_data['sell_price'] * 0.9,2)
#sell_data['amount'] = round(sell_data['target'] * sell_data['qty'])

In [14]:
# Apply filtering with both conditions
filtered = sell_data.loc[
    (sell_data['sell_price'] >= sell_data['price']),colu]

# Sort by %decline   descending
filtered = filtered.sort_values('%decline', ascending=True)

# Format display (if format_dict is defined)
#filtered.style.format(format_dict)
filtered.sample()

,name,sell_date,qty,sell_price,price,low,target,%decline
16,STA,2024-01-03,2500,16.9,12.1,11.9,15.21,-28.4


In [15]:
# Convert columns to interger
filtered["qty"] = filtered["qty"].astype(int)
# Apply formatting
filtered[["qty"]] = filtered[["qty"]].apply(lambda col: col.map(to_integer))
filtered[["sell_price", "price","low", "target"]] = \
filtered[["sell_price", "price","low", "target"]].apply(lambda col: col.map(to_currency))
filtered[["%decline"]] = filtered[["%decline"]].apply(lambda col: col.map(to_percent))
filtered.sample(5)

,name,sell_date,qty,sell_price,price,low,target,%decline
12,WHART,2024-08-26,"5,000",฿10.30,฿9.00,฿9.00,฿9.27,-12.62%
15,BCH,2024-01-04,"2,000",฿22.90,฿13.00,฿12.90,฿20.61,-43.23%
3,RCL,2025-08-08,"3,000",฿31.00,฿27.50,฿27.50,฿27.90,-11.29%
19,SYNEX,2022-12-30,"3,000",฿16.50,฿11.80,฿11.30,฿14.85,-28.48%
5,JMART,2025-08-07,"3,200",฿9.40,฿8.25,฿8.25,฿8.46,-12.23%


In [16]:
# Apply filtering with both conditions
filtered = sell_data.loc[
    (sell_data['sell_date'].dt.year == 2025) & 
    (sell_data['sell_price'] >= sell_data['price']),
    colu]

# Sort by %decline   descending
filtered = filtered.sort_values('%decline', ascending=True)

filtered.sample()

,name,sell_date,qty,sell_price,price,low,target,%decline
1,MCS,2025-08-27,6000,8.4,8.35,8.3,7.56,-0.6


In [17]:
# Convert columns to interger
filtered["qty"] = filtered["qty"].astype(int)
# Apply formatting
filtered[["qty"]] = filtered[["qty"]].apply(lambda col: col.map(to_integer))
filtered[["sell_price", "price","low", "target"]] = \
filtered[["sell_price", "price","low", "target"]].apply(lambda col: col.map(to_currency))
filtered[["%decline"]] = filtered[["%decline"]].apply(lambda col: col.map(to_percent))
filtered

,name,sell_date,qty,sell_price,price,low,target,%decline
5,JMART,2025-08-07,"3,200",฿9.40,฿8.25,฿8.25,฿8.46,-12.23%
3,RCL,2025-08-08,"3,000",฿31.00,฿27.50,฿27.50,฿27.90,-11.29%
2,CPF,2025-08-22,"5,000",฿24.30,฿21.90,฿21.90,฿21.87,-9.88%
4,GVREIT,2025-08-08,"6,000",฿6.70,฿6.20,฿6.20,฿6.03,-7.46%
6,NER,2025-08-06,"5,000",฿4.52,฿4.26,฿4.26,฿4.07,-5.75%
9,IVL,2025-07-30,"1,800",฿22.90,฿21.70,฿21.70,฿20.61,-5.24%
0,ORI,2025-08-28,"5,000",฿2.30,฿2.18,฿2.16,฿2.07,-5.22%
1,MCS,2025-08-27,"6,000",฿8.40,฿8.35,฿8.30,฿7.56,-0.60%


In [18]:
current_path = os.getcwd()
base_path = os.path.dirname(current_path)
ruby_path = os.path.join(base_path, "Ruby")
print(f"Ruby path (ruby_path): {ruby_path}") 

Ruby path (ruby_path): C:\Users\PC1\OneDrive\A5\Ruby


In [19]:
file_name = "watch-list-buys.xlsx"  
output_file = os.path.join(ruby_path, file_name)

# Apply formatting
filtered[["sell_date"]] = filtered[["sell_date"]].apply(lambda col: col.map(to_date))

# Export to Excel
filtered.sort_values('name', ascending=True).to_excel(
    output_file, 
    header=True, 
    index=False
)

### Profit per Month

In [21]:
year = 2025
month = 7
sql = '''
    SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
           sells.price AS sell_price, buys.price AS buy_price, (sells.price - buys.price) AS diff, 
           qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
           (sells.price - buys.price) * qty AS gross, 
           ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, days
    FROM sells JOIN buys ON sells.buy_id = buys.id
    JOIN stocks ON buys.stock_id = stocks.id
    WHERE YEAR(sells.date) = {} AND MONTH(sells.date) = {}
    ORDER BY sells.date DESC, name
'''.format(year, month)

sells_df = pd.read_sql(sql, conpf)
print(sql)


    SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
           sells.price AS sell_price, buys.price AS buy_price, (sells.price - buys.price) AS diff, 
           qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
           (sells.price - buys.price) * qty AS gross, 
           ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, days
    FROM sells JOIN buys ON sells.buy_id = buys.id
    JOIN stocks ON buys.stock_id = stocks.id
    WHERE YEAR(sells.date) = 2025 AND MONTH(sells.date) = 7
    ORDER BY sells.date DESC, name



In [22]:
sells_df

,name,sell_year,sell_month,sell_price,buy_price,diff,qty,sell_amt,buy_amt,gross,pct,days
0,IVL,2025,7,22.9,20.0,2.9,1800,41220.0,36000.0,5220.0,14.50,51
1,WHAIR,2025,7,5.5,4.7,0.8,10000,55000.0,47000.0,8000.0,17.02,75
2,KCE,2025,7,22.9,87.0,-64.1,1000,22900.0,87000.0,-64100.0,-73.68,1391
3,KCE,2025,7,22.4,87.0,-64.6,1000,22400.0,87000.0,-64600.0,-74.25,1386
4,KCE,2025,7,22.2,87.0,-64.8,1000,22200.0,87000.0,-64800.0,-74.48,1312
5,KCE,2025,7,21.7,87.0,-65.3,1000,21700.0,87000.0,-65300.0,-75.06,1310
6,JMT,2025,7,10.2,9.0,1.2,2500,25500.0,22500.0,3000.0,13.33,29
7,KCE,2025,7,21.0,81.0,-60.0,1000,21000.0,81000.0,-60000.0,-74.07,1262
8,KCE,2025,7,19.6,81.0,-61.4,1000,19600.0,81000.0,-61400.0,-75.80,1261


In [23]:
print(f"Monthly Profit Amount: {sells_df.gross.sum():,.2f}")

Monthly Profit Amount: -363,980.00


In [24]:
# Get the current time
current_time = datetime.now()
# Format the time to remove millisAeconds
fmt_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(fmt_time)

2025-08-31 17:01:35
